# Introduction to Function Calling with OpenAI

This notebook demonstrates how to sequentially call various functions using the OpenAI API.

We'll use Python and the OpenAI Python client to interact with the API.

As an example, we'll use the Northwind database to convert natural language queries into SQL.


In [1]:
import json
import sqlite3

import openai
import pandas

In [2]:
# get database schema
def get_schema(db_path="northwind.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema = {}
    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info('{table_name}')")
        columns = cursor.fetchall()
        schema[table_name] = [column[1] for column in columns]
    return schema

In [3]:
# read data from SQLite database given a query
def read_data(query, db_path="northwind.db"):
    conn = sqlite3.connect(db_path)
    df = pandas.read_sql_query(query, conn)
    conn.close()
    return df

In [4]:
# Example SQL query that answers the question: "What is the total revenue for each product in the database?"

sample_query = """ 
SELECT
    p.ProductName,
    SUM(od.Quantity * od.UnitPrice) AS Revenue
FROM  
    Products p
JOIN    
    "Order Details" od
ON
    p.ProductID = od.ProductID
GROUP BY
    p.ProductName
ORDER BY
    Revenue DESC;
"""


df_true = read_data(sample_query)

df_true

,ProductName,Revenue
0,Côte de Blaye,53274482.70
1,Thüringer Rostbratwurst,24630836.96
2,Mishi Kobe Niku,19424638.00
3,Sir Rodney's Marmalade,16654879.80
4,Carnarvon Tigers,12607487.50
...,...,...
72,Tourtière,1509521.85
73,Filo Mix,1421520.80
74,Konbu,1214882.40
75,Guaraná Fantástica,908535.60


In [5]:
QUESTION = """
What is the total revenue for each product in the database? 
Return a table with the product name and total revenue columns. 
Sort by total revenue in descending order."""

messages = [
    {"role": "system",
     "content": """
     The user will ask a question about the database. First, you need to get the schema of the database. Then, you need to read the data from the database.
     Use table names when referring to columns in the database schema. For example, use 'Products.ProductName' instead of just 'ProductName'.
     """},
    {"role": "user",
     "content": QUESTION}
]

functions = [
    # function to get schema of the database
    {
        "type": "function",
        "function": {
            "name": "get_schema",
            "description": "Get the schema of the SQLite database",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        },
    },
    # function to read data from the database
    {
        "type": "function",
        "function": {
            "name": "read_data",
            "description": "Get pandas dataframe from SQLite database given a query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "SQL query to execute"
                    }
                },
                "required": ["query"]
            },
        },
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    tools=functions,
    messages=messages
)
response_message = response.choices[0].message
messages.append(response_message)
tool_call = response_message.tool_calls[0]
function_name = tool_call.function.name
if function_name == "get_schema":
    schema = get_schema()
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": str(schema),
        }
    )
    second_response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        tools=functions,
        messages=messages,
    )

    response_message = second_response.choices[0].message
    messages.append(response_message)
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    if function_name == "read_data":
        # get the query from the tool call
        query = json.loads(tool_call.function.arguments)['query']
        df_openai = read_data(query)

In [6]:
# Verify that the dataframes are the same
# We convert the dataframes to numpy arrays to ignore the differences in column names
(df_true.to_numpy() == df_openai.to_numpy()).all()

False